# Introduction to running PROCESS

A Jupyter notebook to demonstrate usage of the `process` package.

## Motivation
Process is moving away from being a runnable package with a single command-line entry-point to an importable package which can be scripted. This notebook is a good way of demonstrating the functionality of the package, and could provide a better way of working for modellers, who may wish to create their own scripts or notebooks for different tasks.

## Setup
We run the examples in a temporary directory so all the inputs are copied there and the outputs contained there before the directory is removed when the example has finished
running. This keeps the examples directory tidy and does not permanently modify any data files. This use of temporary directories is not needed for regular use of `PROCESS`.

## Basic run of Process
Run Process on an input file using the `SingleRun` class. This outputs an `MFILE.DAT` and an `OUT.DAT`.

In [ ]:
%load_ext autoreload
%autoreload 2

from pathlib import Path

from functions_for_examples import copy_to_temp_dir

from process.main import SingleRun, setup_loggers

# Define project root dir; when running a notebook, the cwd is the dir the notebook is in
PROJ_DIR = Path.cwd().parent

# setup the loggers so that the output is not spammed with model errors/warnings
setup_loggers()

# Define input file name relative to project dir, then copy to temp dir
script_dir = Path("__file__").parent.resolve()
input_rel = script_dir / "data/large_tokamak_IN.DAT"

temp_dir, temp_input_path, temp_dir_path = copy_to_temp_dir(input_rel, PROJ_DIR)

# Run process on an input file in a temporary directory
single_run = SingleRun(temp_input_path.as_posix())
single_run.run()

## Plot summary
Create a summary of the generated `MFILE.DAT` using `plot_proc`.

In [ ]:
from process.io import plot_proc

# plot_proc uses command line arguments of the current process. Jupyter adds command line arguments under the hood causing plot_proc to fail. running plot proc in its own process isolates it from the jupyter command line arguments
# Use the --output-format argument "none" to not save outputs.
# Pdf and png output are also available
plot_proc.main(
    args=["-f", single_run.mfile_path.as_posix(), "--output-format", "none", "--show"]
)

## View key output variables
Using the `MFILE` we generated by running the large tokamak scenario above, we have set some values on the `CostModel` instance and can print them.

In [ ]:
import process.data_structure

# Print some values on the CostModel instance
print(f"Heat transport system: {process.data_structure.cost_variables.c226:.3e} M$")
print(f"Electrical plant equipment: {process.data_structure.cost_variables.c24:.3e} M$")

## Convert to CSV format
This demonstrates how you would read from a PROCESS MFILE and write specified values into a csv using the `mfile_to_csv` function

In [ ]:
from process.io import mfile_to_csv

data_dir = Path("data")

# mfile_to_csv requires two inputs:
# - path to the MFILE
# - .json containing the variable names to include in the csv file

# This routine attempts to find every variable listed in the json file
# in the MFILE and writes the variable name, description and value
# to the output csv.
# Any listed variable that isn't in that MFILE will be skipped.
# The .csv file is saved to the directory of the input file

mfile_to_csv.main(
    args=[
        "-f",
        (data_dir / "large_tokamak_1_MFILE.DAT").as_posix(),
        "-v",
        (data_dir / "mfile_to_csv_vars.json").as_posix(),
    ]
)

In [ ]:
# Clean up
temp_dir.cleanup()